In [4]:
import pickle
import pandas as pd

with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [5]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet')

# Q1

In [22]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

print((sum(y_pred - (sum(y_pred)/len(y_pred)))/len(y_pred))**0.5)

5.27532367217118e-06


# Q2

In [27]:
year = 2022
month = 2
output_file = f'yellow.parquet'

df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
df_result = pd.concat([
    df.ride_id,
    pd.Series(y_pred, name="predicted_duration")
], axis=1)

df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

import os
print(os.path.getsize(output_file))

59994959


# Q3

In [2]:
!jupyter nbconvert --to script 'homework.ipynb'

# Q4

sha256:08ef968f6b72033c16c479c966bf37ccd49b06ea91b765e1cc27afefe723920b

# Q5

In [7]:
year = 2022
month = 3

df = read_data(f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year:04d}-{month:02d}.parquet')

dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

print(sum(y_pred)/len(y_pred))

12.758556818778205
